In [4]:
import pandas as pd 
import numpy as np

In [5]:
import cufflinks as cf
cf.set_config_file(offline=True, world_readable=True, theme='ggplot')
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go

In [6]:
df_train = pd.read_csv("train.csv")
#df_train.info()
#df_train.sample(6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 25 columns):
id        300000 non-null int64
bin_0     300000 non-null int64
bin_1     300000 non-null int64
bin_2     300000 non-null int64
bin_3     300000 non-null object
bin_4     300000 non-null object
nom_0     300000 non-null object
nom_1     300000 non-null object
nom_2     300000 non-null object
nom_3     300000 non-null object
nom_4     300000 non-null object
nom_5     300000 non-null object
nom_6     300000 non-null object
nom_7     300000 non-null object
nom_8     300000 non-null object
nom_9     300000 non-null object
ord_0     300000 non-null int64
ord_1     300000 non-null object
ord_2     300000 non-null object
ord_3     300000 non-null object
ord_4     300000 non-null object
ord_5     300000 non-null object
day       300000 non-null int64
month     300000 non-null int64
target    300000 non-null int64
dtypes: int64(8), object(17)
memory usage: 57.2+ MB


,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
249575,249575,0,0,1,T,Y,Red,Star,Cat,China,...,f2d8eaa42,1,Grandmaster,Warm,h,A,Uk,3,1,0
181200,181200,0,1,0,T,Y,Green,Trapezoid,Cat,Canada,...,151181567,1,Grandmaster,Lava Hot,l,J,Xh,2,12,1
196985,196985,0,0,0,T,Y,Green,Triangle,Cat,Costa Rica,...,a0b386aeb,3,Grandmaster,Cold,g,L,lL,3,3,0
29313,29313,0,1,0,F,Y,Blue,Star,Hamster,Canada,...,0811ee359,1,Novice,Hot,k,E,PQ,3,11,0
40648,40648,0,1,0,T,Y,Green,Polygon,Hamster,Russia,...,2be42caf1,1,Contributor,Cold,l,Q,qv,2,3,1
183644,183644,0,1,0,F,N,Blue,Circle,Lion,Costa Rica,...,a9ce3334c,1,Novice,Hot,l,Z,WE,2,8,0


In [7]:
X = df_train.drop(['target'],axis=1)
y = df_train['target']

## LabelEncoder

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
def label_encoding(df):
    encoder = LabelEncoder()
    df_encoded = df.copy(deep=True)
    
    for col in df_encoded.columns:
        if df[col].dtype == "object":
            df_encoded[col] = encoder.fit_transform(df_encoded[col]) 
        else:
            continue
            
    return df_encoded

In [7]:
en_test = label_encoding(X)
en_test.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,0,1,1,1,5,5,3,...,1686,2175,2,2,1,7,3,136,2,2
1,1,0,1,0,1,1,1,4,3,5,...,650,11635,1,2,3,0,0,93,7,8
2,2,0,0,0,0,1,0,4,4,5,...,1932,8078,1,1,4,7,17,31,7,2
3,3,0,1,0,0,1,2,4,5,0,...,629,6056,1,2,0,8,3,134,2,1
4,4,0,0,0,0,0,2,4,4,0,...,1760,8231,1,2,2,0,17,158,7,8


## Onehot

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
def one_hot_encoding(df):
    df_encoded = df.copy(deep=True)
    
    encoder = OneHotEncoder()
    df_encoded = encoder.fit_transform(df_encoded)
    
    return df_encoded

In [10]:
en_test = one_hot_encoding(X)
en_test

<300000x316461 sparse matrix of type '<class 'numpy.float64'>'
	with 7200000 stored elements in Compressed Sparse Row format>

## Hashing

In [11]:
from sklearn.feature_extraction import FeatureHasher

In [12]:
def hashing_str (df):
    df_encoded = df.copy(deep=True)
    
    encoder = FeatureHasher(input_type='string')
    
    for col in df_encoded.columns:
        df_encoded[col] = df_encoded[col].astype('str')
    
    df_encoded = encoder.transform(df_encoded.values)
    return df_encoded
    
    


In [13]:
en_test = hashing_str(X)
en_test

<300000x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 6483672 stored elements in Compressed Sparse Row format>

## Encoding categories with dataset statistics

In [51]:
def data_counting_encoding (df):
    df_encoded = df.copy(deep=True)
    
    for col in df_encoded.columns:
        if df_encoded[col].dtype == 'object':
            df_encoded[col] = df_encoded[col].astype('category')
            counts = df_encoded[col].value_counts()
            counts = counts.sort_index()
            counts = counts.fillna(0)
            counts += np.random.rand(len(counts))/1000
            df_encoded[col].cat.categories = counts
            
    return df_encoded
            

In [53]:
en_test = data_counting_encoding(X)
en_test.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,0,153535.000908,191633.000794,127341.000275,29855.000116,45979.000263,36942.000416,...,271.000647,19.000760,2,77428.000871,33768.000423,24740.000749,3974.000054,506.000802,2,2
1,1,0,1,0,153535.000908,191633.000794,127341.000275,101181.000117,29487.000037,101123.000338,...,111.000019,13.000842,1,77428.000871,22227.000226,35276.000129,18258.000319,2603.000813,7,8
2,2,0,0,0,146465.000804,191633.000794,96166.000632,101181.000117,101295.000640,101123.000338,...,278.000475,29.000082,1,25065.000655,63908.000614,24740.000749,16927.000584,2572.000068,7,2
3,3,0,1,0,146465.000804,191633.000794,76493.000533,101181.000117,45979.000263,49951.000738,...,203.000254,44.000716,1,77428.000871,60627.000244,28048.000403,3974.000054,2590.000240,2,1
4,4,0,0,0,146465.000804,108367.000747,76493.000533,101181.000117,101295.000640,49951.000738,...,207.000465,35.000233,1,77428.000871,99816.000067,35276.000129,16927.000584,2854.000120,7,8


## Encoding cyclic

In [76]:
def cyclic_encoding(df,columns):
    df_encoded = df.copy(deep=True)
    for col in columns:
        df_encoded[col + '_sin'] = np.sin(2 * np.pi * df_encoded[col] / max(df_encoded[col]))
        df_encoded[col + '_cos'] = np.cos(2 * np.pi * df_encoded[col] / max(df_encoded[col]))
        #df_encoded = df_encoded.drop(columns, axis=1)
        
    return df_encoded.drop(columns, axis=1)

In [77]:
columns = ['day', 'month']
en_test = cyclic_encoding(X,columns)
en_test.head()


,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day_sin,day_cos,month_sin,month_cos
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2,Grandmaster,Cold,h,D,kr,9.749279e-01,-0.222521,0.866025,0.500000
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,1,Grandmaster,Hot,a,A,bF,-2.449294e-16,1.000000,-0.866025,-0.500000
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,1,Expert,Lava Hot,h,R,Jc,-2.449294e-16,1.000000,0.866025,0.500000
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,1,Grandmaster,Boiling Hot,i,D,kW,9.749279e-01,-0.222521,0.500000,0.866025
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,1,Grandmaster,Freezing,a,R,qP,-2.449294e-16,1.000000,-0.866025,-0.500000


## Target encoding

In [91]:
# need a review. Eats all resources. Interruption was required

# def target_encoding(df):
#     df_encoded = df.copy(deep=True)
#     df_encoded['day']=df_encoded['day'].astype('object')
#     df_encoded['month']=df_encoded['month'].astype('object')
    
#     for col in df_encoded.columns:
#         if df_encoded[col].dtype=='object':
#             target = dict (df_encoded.groupby(col)['target'].agg('sum')/df_encoded.groupby(col)['target'].agg('count'))
#             df_encoded[col] = df_encoded[col].replace(target).values
    
#     return df_encoded

In [ ]:
en_test = target_encoding(df_train)
en_test.head()

## K fold target encoding